## Problem statement

# Intial setup of python Libraries for Data and Visualization

In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sea

In [128]:
# Fixing the warning by specifying dtype = object
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 120)

loan_data_df = pd.read_csv('loan.csv', dtype=object)
loan_detail_df = pd.read_excel('Data_Dictionary.xlsx')

# Reading Loan DF and creating Metadata

In [129]:
print("Loan data shape:",loan_data_df.shape)
print("Loan data information")
print(loan_data_df.info())

Loan data shape: (39717, 111)
Loan data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: object(111)
memory usage: 33.6+ MB
None




## Remove Outliners
    Find the median,Q1,Q3 and IQR
    min Outliners = Q1 - 1.5*IQR
    max Outliners = Q3 + 1.5*IQR

## Standardising Value
    Numerical values
    Standardise units
    Scale values if required
    Standardise precision for better presentation of data, e.g. 4.5312341 kgs to 4.53 kgs.
    Remove outliers
    Remove extra characters like common prefix/suffix, leading/trailing/multiple spaces, etc. 
    Standardise case
    Standardise format
## Invalid values
    Encode unicode properly 
    Correct the incorrect data types to the correct data types for ease of analysis.
    Correct values that go beyond range
    Correct values not in the list
    Remove values that don’t belong to a list
    Correct wrong structure
    Validate internal rules

In [130]:
#number of columns in Loan data
print("Number of Columns", loan_data_df.shape[1])
#number of rows in Loan data
print("Number of rows", loan_data_df.shape[0])
#number of duplicate values
print("Duplicate value in Loan data", loan_data_df.duplicated().sum())
#number of columns with only null value
print("Number of columns with null values",(loan_data_df.isnull().all(axis=0).sum()))
#numbers of rows with only null values
print("Number of rows with null values",(loan_data_df.isnull().all(axis=1).sum()))


Number of Columns 111
Number of rows 39717
Duplicate value in Loan data 0
Number of columns with null values 54
Number of rows with null values 0


## Data Cleaning
    Row Cleaning
        Delete summary row if any
        delete incorrect row
        delete empty row
        delete incorrect row
    Checklist for Fixing Columns
        Merge columns for creating unique identifiers if needed
        Split columns for more data: Split address to get State and City to analyse each separately
        Add column names: Add column names if missing
        Rename columns consistently: Abbreviations, encoded columns
        Delete columns: Delete unnecessary columns
        Align misaligned columns: Dataset may have shifted columns

In [131]:
#delete the Loan columns containing only null values
loan_data_df.dropna(axis=1,how='all',inplace=True)

In [132]:
#set to display maximum rows
pd.set_option('display.max_rows', None)
(loan_data_df.isna().sum()).sort_values()

id                                0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths                    0
policy_code                       0
open_acc                          0
pub_rec                           0
revol_bal                         0
addr_state                        0
last_pymnt_amnt                   0
initial_list_status               0
out_prncp                         0
out_prncp_inv                     0
total_pymnt                       0
total_pymnt_inv                   0
total_rec_prncp                   0
total_rec_int                     0
total_rec_late_fee                0
total_acc                         0
recoveries                        0
zip_code                          0
purpose                           0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                        

In [133]:
#dropping off columns with maximum values as null
columns_to_drop = ['desc','mths_since_last_delinq','mths_since_last_record','next_pymnt_d']
loan_data_df.drop(columns_to_drop,axis=1,inplace=True)

## Check the dataframe with unique values

In [90]:
loan_data_df.apply(lambda x: x.nunique()).sort_values()

tax_liens                         1
delinq_amnt                       1
chargeoff_within_12_mths          1
acc_now_delinq                    1
application_type                  1
policy_code                       1
collections_12_mths_ex_med        1
initial_list_status               1
pymnt_plan                        1
term                              2
pub_rec_bankruptcies              3
verification_status               3
loan_status                       3
pub_rec                           5
home_ownership                    5
grade                             7
inq_last_6mths                    9
delinq_2yrs                      11
emp_length                       11
purpose                          14
sub_grade                        35
open_acc                         40
addr_state                       50
issue_d                          55
total_acc                        82
last_pymnt_d                    101
last_credit_pull_d              106
int_rate                    

## Based on the result of nunique method, below columns were identified with same values
1.tax_liens                         1
2.delinq_amnt                       1
3.chargeoff_within_12_mths          1
4.acc_now_delinq                    1
5.application_type                  1
6.policy_code                       1
7.collections_12_mths_ex_med        1
8.initial_list_status               1
9.pymnt_plan                        1

Reading from the loan data dictonary above columns might be important but not this data sets due to common values for all the records, hence deleting these columns

In [134]:
nunique_loan_columns = ['tax_liens','delinq_amnt','chargeoff_within_12_mths','acc_now_delinq','application_type','policy_code'
             ,'collections_12_mths_ex_med','initial_list_status','pymnt_plan']
loan_data_df.drop(columns=nunique_loan_columns,axis=1,inplace=True)

In [135]:
#on further analysis of columns, we can drop the 
# zip_code - Does not have the full zip-code and the xx - value code be either same or different
# url - Does not provide relevant information
# member_id - this dataframe contain unique values only, so we can drop it as well
# emp_title - the are text and does not contribute 
nonrelevant_loan_column_values = ['zip_code','url','member_id','emp_title']
loan_data_df.drop(columns=nonrelevant_loan_column_values,axis=1,inplace=True)

## Standarization of Columns

In [136]:
# updating the NAN value available in different columns to clean values
# pub_rec_bankruptcies - update the values with 'Not Available'
# emp_length - format the employee length correctly to Numerics values
loan_data_df.pub_rec_bankruptcies.fillna('Not Available',inplace=True)
loan_data_df.emp_length.fillna('0',inplace=True)
loan_data_df['emp_length'] = loan_data_df['emp_length'].apply(lambda x : x.replace('years','')
                                             .replace('+','')
                                             .replace('< 1','0.5')
                                             .replace('year',''))

In [137]:
# Now that we have filtered out -irrelevant columns, let check and clean the columns values
# int_rate - remove % symbol, check dtype if object convert it to float
# revol_util - remove % symbol, check dtype if object convert it to float
# term - remove months from the term value
# emp_length - formatting the columns valye
loan_data_df['int_rate'] = loan_data_df['int_rate'].str.rstrip('%')
loan_data_df['revol_util'] = loan_data_df['revol_util'].str.rstrip('%')
loan_data_df['term'] = loan_data_df['term'].str.rstrip('months')


## Update the amount columns to numeric values

In [138]:
# updating object type to numeric for amount columns
numeric_columns = ['loan_amnt','funded_amnt','funded_amnt_inv','installment','annual_inc','revol_bal',
                   'total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int']
loan_data_df[numeric_columns] = loan_data_df[numeric_columns].apply(pd.to_numeric)

In [143]:
loan_data_df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies


## Find the outliners in loan_amount

Find the median,Q1,Q3 and IQR </br>
min Outliners = Q1 - 1.5*IQR </br>
max Outliners = Q3 + 1.5*IQR </br>

In [140]:
# common function to find the outliers
# Find the median,Q1,Q3 and IQR
# min Outliners = Q1 - 1.5IQR
# max Outliners = Q3 + 1.5IQR

def findOutliers(df, column ):
    quat1 = df[column].quantile(0.25)
    quat3 = df[column].quantile(0.75)
    iqr = quat3 - quat1
    min_outliner = loan_q1 - (1.5*loan_q1)
    max_outliner = loan_q3 - (1.5*loan_q3)

    return {'quat1': quat1, 'quat3': quat3, 'iqr': iqr, 'min_outliner': min_outliner, 'max_outliner': max_outliner}

def compareOutliers(df,column):
    
    # box plot before dropping outliers
    #fig, p = plot.subplots(1,2,figsize=(14, 3))
    splot1 = sea.boxplot(x = df[column])
    #splot1.set_title('Plot ['+ column + '] - Original')
    new_df = df[df[column] < findOutliers(df, column)['max_outliner']]
    # box plot after dropping outliers    
    splot2 = sea.boxplot(x = new_df[column])
    #splot2.set_title('Plot [' + column + '] - Post Outlier Treatment')
    #plot.tight_layout()
    plot.show()
    

def deleteMaxOutliers(df,column):
    new_df = df[df[column] < findOutliers(df, column)['max_outliner']]
    return new_df

def deleteMinOutliers(df,column):
    new_df = df[df[column] > findOutliers(df, column)['min_outliner']]
    return new_df


## Removing Outliers

In [141]:
# columns subject to outliers are
# loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti
# removing outliers from above columns for better analysis

outliers_columns = ['loan_amnt','funded_amnt','funded_amnt_inv','int_rate','installment','annual_inc','dti']
for column in outliers_columns:
    #compareOutliers(loan_data_df,column)
    loan_data_df = deleteMaxOutliers(loan_data_df,column)


In [142]:
loan_data_df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies


## Derived columns

In [22]:
# deriving year and month from loan issue dates so that, we can generate new dervied columns by binning the values

loan_data_df['issue_d'] = pd.to_datetime(loan_data_df.issue_d, format='%b-%y')
loan_data_df['issued_year'] = pd.DatetimeIndex(loan_data_df['issue_d']).year
loan_data_df['issued_month'] = pd.DatetimeIndex(loan_data_df['issue_d']).month

In [ ]:
# binning the month columns to create new derieved cloumns 'quarters'

loan_data_df[]
